In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

#Data processing
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
import datetime
import math
from sklearn.preprocessing import LabelEncoder
import statistics
from sklearn.metrics import mean_squared_error
from tqdm import tqdm
from itertools import product
from math import sqrt
import json
from sklearn.preprocessing import PolynomialFeatures

#Models
from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense, LSTM, Dropout,Reshape
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import clone_model
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

#Graphs
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import periodogram
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_pacf

In [ ]:
sample=pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')
test=pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
train=pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')

In [ ]:
train.describe()

In [ ]:
X=train.drop(columns=['Transported'])
y=train['Transported']
# X is Feature, Y is Target

In [ ]:
X.columns

In [ ]:
X.head()

In [ ]:
X.info()

In [ ]:
X.dtypes

In [ ]:
nulls=[]
unique=[]

for col in X.columns:
    nulls.append(X[col].isnull().sum())
    unique.append(X[col].nunique())
    
summary=pd.DataFrame(data={'Columns':X.columns,'No of null':nulls,'No of unique values':unique})
summary

In [ ]:
X.corr()

In [ ]:
#sns.pairplot(X.select_dtypes(include=['float64']))

First lets split into deck num side 

In [ ]:
# I'll check on how to do string formatting
train_deck = [] 
train_count = []
test_deck = [] 
test_count = []

deck = ['Cabin','Value Counts']

# count_decks()
# pd.set_option('display.max_rows', None)

# print(, test['Cabin'].value_counts(), sep ='/t')

# For train

train_deck.append(train['Cabin'])
train_count.append(train['Cabin'].value_counts())
    
summary_train = pd.DataFrame(data = {deck[0] : train_deck , deck[1]: train_count})
print(summary_train)

# For test

test_deck.append(test['Cabin'])
test_count.append(test['Cabin'].value_counts())

summary_test = pd.DataFrame(data = {deck[0]:test_deck, deck[1]: test_count})
summary_test

Saw above code - 



Train & Test:

Generally, Age, FoodCourt, SPA, VRDeck shows best positive correlation with one another

For negative correlation, 






Lasso Regression, L1/L2 Regression

In [ ]:
test.describe()

In [ ]:
test.corr()

In [ ]:
missing = test['Age'].isna()
feature = test.columns

def blank_data(file, locate):
    return test.loc[missing, :]
blank_data(test, missing)

In [ ]:
missing = test['Age'].isna()

# def blank_data(file, locate):
#     return test.loc[missing]  
# blank_data(test, missing)

test.dropna(how = 'any')
    
test

# TBC

**Arden's Edit:*

Step 1.1: Define X & Y

Step 1.2: Split Cabin into 3 separate columns

Step 1.3: Splitting dataset into train and validation

Step 2.1: Seeing which columns in X_train are in need of imputation/deletion

Find out how to impute a catagorical column based on other columns

In [ ]:
X=train.drop(columns=['Transported'])
Y=train['Transported']
# X is Feature, Y is Target

X_test = test

In [ ]:
X["Cabin"]
X[["Deck","Num","Side"]]=X["Cabin"].str.split("/",expand = True)
X.drop("Cabin",axis=1,inplace=True)
X.drop("Name",axis=1,inplace=True)


In [ ]:
X.head(20)

I noticed that Eur

In [ ]:
from sklearn.model_selection import train_test_split 
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.33, random_state=42)


In [ ]:
#sns.heatmap(X_train.isna())

In [ ]:
X_train.isna().sum()

Imputation done by Aus.
 
According to ChatGPT take, 
1. Generally impute training dataset, then apply imputation method to test dataset
2. Missing datas are considered Missing At Random (MAR). 
Multiple Imputation, Regression, KNN, EM, Hot Deck were recommended. 
Among them, Regression is recommended although you must consider other techniques at own discretion.

My take: 
1. Generally, Age, FoodCourt, SPA, VRDeck shows best positive correlation with one another.
For negative correlation, train and test differs. It is seen that RoomService, VRDeck, FoodCourt shows best and consistent negative correlation with one another.
Perhaps they are multivariate data.

2. I would go impute Age, FoodCourt, SPA, VRDeck, RoomService , ShoppingMall. Removed some columns as to explore for any surprising results.
Currently, used Regression Imputation.

Not sure why info() and other things are not updated, despite dropping, adding etc.....

In [ ]:
X_train.sort_values(by=['Age'], ascending = True)
X_train[['Age', 'FoodCourt','Spa','VRDeck']].head(10)


In [ ]:
X_train.sort_values(by = ['Age'], ascending = True)
X_train[['RoomService', 'VRDeck', 'FoodCourt']].head(10)

In [ ]:
# Checking of shape to ensure train and valid features and target corresponds correctly.

# X_train.shape # (5824,15)
# X_valid.shape # (2869,15)
# y_train.shape # (5824,)
# y_valid.shape # (2869,)

# It's correct

In [ ]:
X_train.head(len(X_train))

In [ ]:
# Trying Regression Imputation.
# Assuming X is your DataFrame
X_train[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']] = X_train[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']].interpolate(method='linear')
X_train

In [ ]:
# Evaluating the Correlation Metrics to a model. Feel free to put in yourthought
delete =X_train.drop(columns = ['PassengerId', 'HomePlanet','CryoSleep','Destination','VIP', 'Deck', 'Num', 'Side'])
X_train.info()
# Imputed the amenities.

Finding causation of an event. 
I would think that Random Forest/Bagged Tree, LogisticsRegression are possible to use.

In [ ]:
random_forest = RandomForestRegressor()

Logistics Regression. Goal is to look at how to encode accordingly

In [ ]:
#sns.heatmap(X_test.isna())

now will proceed to impute the validation dataset using Austin's imputation method.

In [ ]:

X_valid[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']] = X_valid[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']].interpolate(method='linear')
X_valid

now will do the same to X_test

In [ ]:

X_test[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']] = X_test[['Age', 'FoodCourt', 'ShoppingMall','Spa', 'VRDeck', 'RoomService']].interpolate(method='linear')
X_test

i think ill remove the name column as it is unnecessary

In [ ]:
X_test.drop("Name",axis=1,inplace=True)
X_test

In [ ]:
sns.heatmap(X_test.isna())

In [ ]:
X_train[X_train["VIP"]==False]["Age"].mean()

In [ ]:
X_train[X_train["VIP"]==True]["Age"].mean()

In [ ]:
sns.displot(X_train[X_train["VIP"]==False]["Age"])

In [ ]:
X_train["Age"]

In [ ]:
sns.countplot(x="VIP",hue="CryoSleep",data=X_train)

In [ ]:
count = len(X_train[(X_train['VIP'] == True) & (X_train['CryoSleep'] == False)])
count

In [ ]:
X_train["VIP"].isna()=X_train[X_train["CryoSleep"]==True]["VIP"].fillna(True)